In [1]:
  !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr xgboost -q
  !pip install rouge
  !pip install --upgrade rouge-score

  from transformers import BartForConditionalGeneration
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  import pandas as pd
  import numpy as np
  import torch
  from sklearn.metrics import mean_squared_error
  from rouge_score import rouge_scorer
  from transformers import PegasusForConditionalGeneration, AutoTokenizer
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  from rouge import Rouge
  from datasets import load_dataset
  from google.colab import drive
  import nltk
  import xgboost as xgb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:0

In [2]:
# Initialize the Bart tokenizer and model
bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')

# Initialize the pegasus tokenizer and model
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:

df_bart_train= pd.read_csv('/content/drive/MyDrive/GP data/Bart_Train_Summaries(0-11490).csv')
df_pegasus_train=pd.read_csv('/content/drive/MyDrive/GP data/Pegasus_Summaries_Train(0-11490).csv')
df_bart_test=pd.read_csv('/content/drive/MyDrive/GP data/Bart_Summaries_Test_[0-11490] (1).csv')
df_pegasus_test=pd.read_csv('/content/drive/MyDrive/GP data/Pegasus_Test_Summaries_0-11491.csv')

In [9]:
import nltk
import string

# download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# define function to clean text
def clean_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation+ '.'))
    # convert to lowercase
    text = text.lower()
    # tokenize text into words
    words = nltk.word_tokenize(text)
    # remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    words = [word for word in words if word not in stopwords]
    # join words back into a string
    text = ' '.join(words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
# apply cleaning function to summary column
df_bart_train['summary'] = df_bart_train['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_bart_train['highlights'] = df_bart_train['highlights'].apply(clean_text)

In [11]:
# apply cleaning function to summary column
df_pegasus_train['summary'] = df_pegasus_train['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_pegasus_train['highlights'] = df_pegasus_train['highlights'].apply(clean_text)

In [12]:
# apply cleaning function to summary column
df_bart_test['summary'] = df_bart_test['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_bart_test['highlights'] = df_bart_test['highlights'].apply(clean_text)

In [13]:
# apply cleaning function to summary column
df_pegasus_test['summary'] = df_pegasus_test['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_pegasus_test['highlights'] = df_pegasus_test['highlights'].apply(clean_text)

In [48]:
x_bart_train = []
for i, summary in df_bart_train['summary'].items():
    encoded_summary=bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_bart_train.append((i, encoded_summary.numpy().squeeze()))

In [49]:
x_pegasus_train = []
for i, summary in df_pegasus_train['summary'].items():
    encoded_summary=bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_pegasus_train.append((i, encoded_summary.numpy().squeeze()))

In [50]:
y_train = []
for i, summary in df_bart_train['highlights'].items():
    encoded_summary = bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    y_train.append((i, encoded_summary.numpy().squeeze()))

In [51]:
x_bart_test = []
for i, summary in df_bart_test['summary'].items():
    encoded_summary =  bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_bart_test.append((i, encoded_summary.numpy().squeeze()))

In [52]:
x_pegasus_test = []
for i, summary in df_pegasus_test['summary'].items():
    encoded_summary = bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_pegasus_test.append((i, encoded_summary.numpy().squeeze()))

In [64]:
y_test = []
for i, summary in df_pegasus_test['highlights'].items():
    encoded_summary = bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    y_test.append((i, encoded_summary.numpy().squeeze()))

In [54]:
# Convert the input data to numpy arrays
x_bart_train = np.array(x_bart_train)
y_train = np.array(y_train)
x_bart_test = np.array(x_bart_test)
y_test = np.array(y_test)
x_pegasus_train = np.array(x_pegasus_train)
x_pegasus_test = np.array(x_pegasus_test)

<ipython-input-54-a33e8d47f684>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_bart_train = np.array(x_bart_train)
<ipython-input-54-a33e8d47f684>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array(y_train)
<ipython-input-54-a33e8d47f684>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_bart_test = np.array(x_

In [65]:
x_bart_train_indices = np.array([t[0] for t in x_bart_train])
x_bart_train_summaries = np.array([t[1] for t in x_bart_train])
y_train_indices = np.array([t[0] for t in y_train])
y_train_summaries = np.array([t[1] for t in y_train])
x_pegasus_train_indices = np.array([t[0] for t in x_pegasus_train])
x_pegasus_train_summaries = np.array([t[1] for t in x_pegasus_train])

In [66]:
x_bart_test_indices = np.array([t[0] for t in x_bart_test])
x_bart_test_summaries = np.array([t[1] for t in x_bart_test])
y_test_indices = np.array([t[0] for t in y_test])
y_test_summaries = np.array([t[1] for t in y_test])
x_pegasus_test_indices = np.array([t[0] for t in x_pegasus_test])
x_pegasus_test_summaries = np.array([t[1] for t in x_pegasus_test])

In [58]:
# Stack the BART and Pegasus summaries
predictions_train = np.column_stack((x_bart_train_summaries, x_pegasus_train_summaries))
predictions_test = np.column_stack((x_bart_test_summaries[0:11489], x_pegasus_test_summaries))

In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [68]:
# Train a RandomForestRegressor on BART summaries
rf_base = RandomForestRegressor(n_estimators=1)
rf_base.fit(predictions_train, y_train_summaries)

RandomForestRegressor(n_estimators=1)

In [69]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

# Train a KNeighborsRegressor on Pegasus summaries
mlp_base = MLPRegressor()
mlp_base.fit(predictions_train, y_train_summaries)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor()

In [ ]:
# Step 5: Create a new DataFrame to store the predictions
predictions = pd.DataFrame({"rf_preds": rf_base.predict(predictions_test[0:5000]).ravel(), "mlp_preds": mlp_base.predict(predictions_test[0:5000]).ravel()})

In [73]:
y_test_flat = y_test_summaries[0:5000].ravel()

In [74]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

# Initialize base estimators
estimators = [
    ('rf',  RandomForestRegressor()),
    ('mlp', MLPRegressor()),
]


# Initialize meta-model with base estimators
meta_model =GradientBoostingRegressor(n_estimators=100)
meta_model.fit(predictions, y_test_flat)

GradientBoostingRegressor()

In [75]:
test_rf_preds=rf_base.predict(predictions_test[5000:]).ravel()
test_mlp_preds=mlp_base.predict(predictions_test[5000:]).ravel()

In [76]:
# Step 5: Create a new DataFrame to store the predictions
test_predictions = pd.DataFrame({"rf_preds": test_rf_preds, "mlp_preds":test_mlp_preds})

In [77]:
ensemble_preds = meta_model.predict(test_predictions)

In [78]:
ensemble_preds = ensemble_preds.reshape((6489,-1))

In [81]:
ensemble_preds_text = []
for  i in range(len(ensemble_preds)):
    summary_ids = ensemble_preds[i]
    summary_ids = summary_ids.astype(int)
    summary = bart_tokenizer.decode(summary_ids, skip_special_tokens=True, max_length=100)
    ensemble_preds_text.append(summary)

In [ ]:
y_test_text = []
for i in range(len(y_test)):
    encoded_summary = y_test[i][1]
    decoded_summary = bart_tokenizer.decode(encoded_summary.squeeze(), skip_special_tokens=True, max_length=100)
    y_test_text.append( decoded_summary)

In [ ]:
import sys

sys.setrecursionlimit(10000) # set the recursion limit to 10000

rouge = Rouge()
batch_size = 500
num_batches = (len(ensemble_preds_text) + batch_size - 1) // batch_size # round up to nearest integer

scores = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i+1) * batch_size, len(ensemble_preds_text))
    batch_pred_text = ensemble_preds_text[start_idx:end_idx]
    batch_ref_text = y_test_text[start_idx:end_idx]

    batch_scores = []
    for j in range(len(batch_pred_text)):
        reference = str(batch_ref_text[j])
        hypothesis = str(batch_pred_text[j])
        if not reference:
            score = {'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}
        else:
            score = rouge.get_scores(hypothesis, reference)
        batch_scores.append(score)

    scores.extend(batch_scores)

In [ ]:
# Get the three ROUGE scores for each summary and calculate the average
rouge_1_scores = [score[0]["rouge-1"]["f"] for score in scores]
rouge_l_scores = [score[0]["rouge-l"]["f"]for score in scores]
average_rouge_1 = sum(rouge_1_scores) / (len(scores))
average_rouge_L = sum(rouge_l_scores) / (len(scores))

In [ ]:
# Save the ROUGE scores to a CSV file
rouge_scores_df = pd.DataFrame({
    "rouge-1": rouge_1_scores,
    "rouge-l": rouge_l_scores,
    "Average-1": average_rouge_1,
    "Average-L": average_rouge_L,
})
rouge_scores_df.to_csv("Bart&Peg_Gradient_Rouge_Scores.csv", index=False)